# Working with a database

This week's assignment has a few basic steps.  First, we're going to pull some data down off the internet and store it into our MySQL database.  Make sure that you use your username as part of the table name as show in the examples so that you don't create a problem for other students.

Then, we'll merge that with some data already in the database and calculate a few results.  When it comes to calculating the results, you can do so either with SQL or with Pandas operations.


## PART 1: Setup your database connection and table name

In the code below, change the value of the variable `MYTABLE` to use your own username rather that `'pboal'`

You can then use `MYTABLE` in the rest of your code to reference that table name.

In [ ]:
import os
import pymysql
from sqlalchemy import create_engine
import pandas as pd

In [ ]:
#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!
# First, put in your user name below
#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!

MYTABLE='dhumphrey420' + '_data'


# Then, create a connection to the same database, using the 
# same user name and password in the `mysql-examples.ipynb` 
# file.  Use conn as the database connection variable.

host = 'slucor2020.cgdcoitnku0k.us-east-1.rds.amazonaws.com'
port = '3306'
user = 'slucor2020'
password = 'SLUcor2020'
database = 'hds5210'

conn = create_engine('mysql+pymysql://' + 
                     user + ':' + 
                     password + '@' + 
                     host + '/' + 
                     database, echo=False)


In [9]:
assert(MYTABLE != 'paulboal_data')
assert(conn.connect())

In [10]:
MYTABLE

'dhumphrey420_data'

## PART 2: Bring in outside data

Grab the data from this URL and put it into a database table named with your `username_data`.

http://dhcs-chhsagency.opendata.arcgis.com/datasets/8e4f3a0c75b9424d888d11c1f949cc32_0.csv

By the end of your cell, the table should be created.  The tests are going to verify that the table exists and looks right.

In [11]:
url = 'http://dhcs-chhsagency.opendata.arcgis.com/datasets/8e4f3a0c75b9424d888d11c1f949cc32_0.csv'

### SOLUTION GOES HERE
df= pd.read_csv(url)
df
dxyz = pd.read_sql_query('SELECT * FROM ' + MYTABLE, conn)

In [ ]:
pd.read_sql_query('SHOW TABLES', conn)

In [13]:
tables = {}

for n,t in pd.read_sql_query("SHOW TABLES", conn).iterrows():
    name = str(t['Tables_in_hds5210'])
    print(name)
    tables[name] = pd.read_sql("SELECT * FROM "+name,conn)

Medicaid_EP_Hospital_Type
NPI
Population
Provider_Name
Test
a85531647_data
aavila_data
aavila_data_NPI
aavila_data_new
aavila_data_year
aavila_datax
abbieslu
abbieslu_NPI
abbieslu_ipv
abbieslu_npi
abbieslu_year
achenliam
achenliam_data
achenliam_ipv
apuryear_data
arefehesmaeilpour_data
ashley_data
btabman_data
corona_counts
county_population
dhumphrey420_data
ed_visits
hospital_services
hospitals
jdgendron_data
jmccar23_data
kaylaschmidt_data
kenny0415_data
khalafh_data
kodurus_data
lchen_data_ipv
liannecota_data
liannecota_newdata
paulboal_data
petersluu_data
population
ppp_data
qichen_data
sam
sampele
sneha-koduru_data


ProgrammingError: (pymysql.err.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '-koduru_data' at line 1")
[SQL: SELECT * FROM sneha-koduru_data]
(Background on this error at: http://sqlalche.me/e/f405)

In [ ]:
data = tables['dhumphrey420_data'].merge(tables['population'])

In [ ]:
tables['dhumphrey420_data']

In [ ]:
dxyz

In [ ]:
dxyz.columns

In [59]:
dxyz = pd.read_sql_query('SELECT * FROM ' + MYTABLE, conn)
assert(dxyz.shape == (328,19))
assert(list(dxyz.columns) == ['Provider_Name', 'NPI', 'CCN',
       'Medicaid_EP_Hospital_Type', 'Street_Address', 'City', 'County',
       'State', 'Zip_Code', 'Payment_Year_Number', 'Program_Type',
       'Total_payments', 'Last_Program_Year', 'Last_Payment_Year',
       'Last_Payment_Criteria', 'Most_Recent_Disbursement_Amount', 'LONGITUDE',
       'LATITUDE', 'FID'])

In [67]:
join_table= pd.read_sql_query("""
  SELECT d.*, p.*
  FROM 
    dhumphrey420_data d JOIN
    population p ON d.Zip_Code = p.zipcode
    WHERE
    d.Zip_Code = p.zipcode
""", conn)

In [70]:
join_table[['Zip_Code', 'zipcode']]

,Zip_Code,zipcode
0,90804,90804
1,91754,91754
2,91773,91773
3,91010,91010
4,91402,91402
...,...,...
82,91776,91776
83,91367,91367
84,90242,90242
85,91505,91505


In [71]:
match = join_table['Zip_Code'] == 'zipcode'

/opt/tljh/user/lib/python3.6/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [73]:
match.count

<bound method Series.count of 0     False
1     False
2     False
3     False
4     False
      ...  
82    False
83    False
84    False
85    False
86    False
Name: Zip_Code, Length: 87, dtype: bool>

In [78]:
join_table['population'].max()

105549

In [80]:
row=join_table.loc[join_table['population'] == 105549]

In [82]:
row['NPI']

75    1194016923
Name: NPI, dtype: int64

## PART 3: Combine with other data in the database

In the database, there is an existing table called `population`.  We want to merge the DHCS datafile loaded above with the population data available in this other database table  The tables can be merged on `MYTABLE`'s `Zip_Code` field and `population`'s `zipcode` field.

Note that not all `Zip_Codes` from your downloaded file have to be in the `population` table.  If they aren't, then I want you to eliminate the non-matching records.  That is, only keep the records that have a matching ZIP code in both sets of data.

Answer the question:
Which providers are located in the zipcode with the largest population?

Put your answer in the form `answer = ['a', 'list', 'of', 'NPI', 'like', '1593042103]`

In [86]:
### SOLUTION GOES HERE
join_table= pd.read_sql_query("""
  SELECT d.*, p.*
  FROM 
    dhumphrey420_data d JOIN
    population p ON d.Zip_Code = p.zipcode
    WHERE
    d.Zip_Code = p.zipcode
""", conn)
join_table['population'].max()
row=join_table.loc[join_table['population'] == 105549]
row['NPI']
answer = ['1194016923']

In [87]:
assert(type(answer) == list)
assert(answer == ['1194016923'])

In [89]:
dxyz.columns

Index(['Provider_Name', 'NPI', 'CCN', 'Medicaid_EP_Hospital_Type',
       'Street_Address', 'City', 'County', 'State', 'Zip_Code',
       'Payment_Year_Number', 'Program_Type', 'Total_payments',
       'Last_Program_Year', 'Last_Payment_Year', 'Last_Payment_Criteria',
       'Most_Recent_Disbursement_Amount', 'LONGITUDE', 'LATITUDE', 'FID'],
      dtype='object')

In [92]:
join_table

,Provider_Name,NPI,CCN,Medicaid_EP_Hospital_Type,Street_Address,City,County,State,Zip_Code,Payment_Year_Number,...,LONGITUDE,LATITUDE,FID,zipcode,population,median_age,total_males,total_females,total_households,avg_household
0,LONG BEACH MEMORIAL MEDICAL,1720371065,50727,Acute Care Hospitals,1720 TERMINO AVE,LONG BEACH,Los Angeles,CA,90804,2,...,-118.145915,33.788689,20,90804,40311,29.8,19686,20625,14556,2.70
1,AHMC GARFIELD MEDICAL CENTER LP,1730171265,50737,Acute Care Hospitals,525 N GARFIELD AVE,MONTEREY PARK,Los Angeles,CA,91754,4,...,-118.122909,34.067656,21,91754,32742,42.9,15620,17122,11193,2.92
2,PRIME HEALTHCARE SERVICES - SAN DIMAS LLC,1740447945,50588,Acute Care Hospitals,1350 W COVINA BLVD,SAN DIMAS,Los Angeles,CA,91773,4,...,-117.834284,34.099075,28,91773,33119,42.5,15737,17382,11941,2.73
3,City of Hope National Medical Center,1750358297,50146,Acute Care Hospitals,1500 DUARTE RD,DUARTE,Los Angeles,CA,91010,1,...,-117.971512,34.132255,29,91010,26074,38.1,12461,13613,7972,3.20
4,DEANCO HEALTHCARE LLC,1750365375,50704,Acute Care Hospitals,14850 ROSCOE BLVD,PANORAMA CITY,Los Angeles,CA,91402,4,...,-118.456715,34.221469,30,91402,69817,30.1,35095,34722,18194,3.81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,"AHMC SAN GABRIEL VALLEY MEDICAL CENTER, LP",1275720377,50132,Acute Care Hospitals,438 W LAS TUNAS DR,SAN GABRIEL,Los Angeles,CA,91776,2,...,-118.105202,34.102491,313,91776,38475,39.0,18751,19724,11776,3.24
83,KAISER FOUNDATION HOSPITALS,1295880912,50677,Acute Care Hospitals,5601 DE SOTO AVE,WOODLAND HILLS,Los Angeles,CA,91367,2,...,-118.588180,34.170412,315,91367,39499,41.0,19171,20328,16697,2.35
84,COUNTY OF LOS ANGELES AUDITOR CONTROLLER,1336154020,50717,Acute Care Hospitals,7601 E IMPERIAL HWY,DOWNEY,Los Angeles,CA,90242,2,...,-118.156423,33.926359,325,90242,43497,31.6,21207,22290,12687,3.41
85,PROVIDENCE HEALTH SYSTEM-SOUTHERN CALIFORNIA,1336173269,50235,Acute Care Hospitals,501 S BUENA VISTA,BURBANK,Los Angeles,CA,91505,4,...,-118.326997,34.155912,327,91505,30778,38.9,15121,15657,13180,2.33


In [90]:
by_hospital= dxyz.groupby(['Zip_Code','Medicaid_EP_Hospital_Type'])['Total_payments'].sum().reset_index()

In [91]:
by_hospital

,Zip_Code,Medicaid_EP_Hospital_Type,Total_payments
0,90004,Acute Care Hospitals,1292117
1,90012,Acute Care Hospitals,3840496
2,90015,Acute Care Hospitals,5580824
3,90017,Acute Care Hospitals,4195996
4,90018,Acute Care Hospitals,1553491
...,...,...,...
290,96104,Acute Care Hospitals,184615
291,96122,Acute Care Hospitals,515430
292,96130,Acute Care Hospitals,1323384
293,96150,Acute Care Hospitals,350678


In [ ]:
join_table= pd.read_sql_query("""
  SELECT b.*, p.*
  FROM 
     d JOIN
    population p ON d.Zip_Code = p.zipcode
    WHERE
    d.Zip_Code = p.zipcode
""", conn)

## PART 4: Total by hospital type

This is a multistep process:
* From our downloaded data file, compute the `Total payments` per ZIP code and Medicaid EP Hospital Type.
* Then merge that with the `population` data to compute a `Total payments` per person.
* Then average that across all of the `Medicaid EP Hospital Types` to get an average per persona payment for these type of hospital.

Your answer should be in structure of a data frame with at least two columns:
* Medicaid_EP_Hospital_Type
* Avg_Pay_per_Capita

In [95]:
### SOLUTION GOES HERE
answer = pd.read_sql_query("""
Select Medicaid_EP_Hospital_Type, AVG(pay_per_capita) as Avg_Pay_per_Capita
From
(Select dhumphrey420_data.Medicaid_EP_Hospital_Type, dhumphrey420_data.Zip_Code, dhumphrey420_data.sum_tot_payments/population.population as pay_per_capita 
FROM
(SELECT Zip_Code, Medicaid_EP_Hospital_Type, SUM(Total_payments) as sum_tot_payments FROM
dhumphrey420_data
group by
Zip_Code, Medicaid_EP_Hospital_Type) dhumphrey420_data
JOIN
population population
ON
dhumphrey420_data.Zip_Code = population.zipcode) d
Group By
Medicaid_EP_Hospital_Type
""", conn)
answer

,Medicaid_EP_Hospital_Type,Avg_Pay_per_Capita
0,Acute Care Hospitals,17974.362796
1,Children's Hospitals,111.559033


In [96]:
assert(type(answer) == pd.core.frame.DataFrame)
assert(round(answer.query("Medicaid_EP_Hospital_Type == 'Acute Care Hospitals'")['Avg_Pay_per_Capita'].sum(),3) == 17974.363)
assert(round(answer.query("Medicaid_EP_Hospital_Type == 'Children\\'s Hospitals'")['Avg_Pay_per_Capita'].sum(),3) == 111.559)
